### Persistence images from distance matrix

In [17]:
# import libraries
import numpy as np
import re
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import traceback
import pandas as pd


from ripser import Rips
from persim import PersistenceImager

import glob
import pickle
import geopandas as gpd
import dcor
import pandas as pd

In [4]:
# remove warnings
import warnings
warnings.filterwarnings("ignore")

In [8]:
import ripser
import persim

def diagram_sizes(dgms):
    return ", ".join([f"|$H_{i}$|={len(d)}" for i, d in enumerate(dgms)])

In [21]:
# Define constant variables
DATA_DIR = "/Users/h6x/ORNL/git/modeling-ideas/overdose modeling for entire country/data/processed data/selected coordinates for each state - percentiles(below 90th)- all variables"
RESULTS_DIR = "/Users/h6x/ORNL/git/modeling-ideas/overdose modeling for entire country/results/persistence images/below 90th percentile/h1h0/distance matrix 16 channels npy"
VARIABLES = ['EP_POV','EP_UNEMP','EP_PCI','EP_NOHSDP','EP_UNINSUR','EP_AGE65','EP_AGE17','EP_DISABL','EP_SNGPNT','EP_LIMENG','EP_MINRTY','EP_MUNIT','EP_MOBILE','EP_CROWD','EP_NOVEH','EP_GROUPQ']
PERSISTENCE_IMAGE_SHAPE = (310, 310)
NUMBER_OF_VARIABLES = 16
PERSISTENCE_IMAGE_PARAMS = {
    'pixel_size': 0.001,
    'birth_range': (0.0, 0.31),
    'pers_range': (0.0, 0.31),
    'kernel_params': {'sigma': 0.00004}
}

In [26]:
# imoporting SVI data for the entire US(county level) 
us_svi = gpd.read_file('/Users/h6x/ORNL/git/modeling-ideas/overdose modeling for entire country/data/processed data/svi with hepvu/2018/SVI 2018 with HepVu census tracts/SVI2018_US_census_with_opioid_indicators.shp')

In [11]:
# Function to get the list of folders in a specified location
def get_folders(location):
    return [name for name in os.listdir(location) if os.path.isdir(os.path.join(location, name))]

# Get the list of state folders
states = get_folders(DATA_DIR)

In [12]:
# Create a folder for each variable if it does not exist
for variable in VARIABLES:
    os.makedirs(os.path.join(RESULTS_DIR, variable), exist_ok=True)
print('Done creating folders for each variable')

Done creating folders for each variable


In [37]:
# Loop through each state
for state in tqdm(states, desc="Processing states"):
    print('Processing:', state)

    try:
        # Load data from pickle files into a dictionary
        data = {}

        for file in glob.glob(os.path.join(DATA_DIR, state, '*.pkl')):
            with open(file, 'rb') as f:
                
                # Extract the last 20 characters of the file name
                extracted_words = file[-20:]

                # Search for numbers in the extracted string
                match = re.search(r'(\d+)', extracted_words)
                if match:
                    extracted_number = match.group(1)
                    # Load the pickle file data into the dictionary
                    data[extracted_number] = pd.read_pickle(f)
                else:
                    print("No number found in the string.")

                 # Process each county (FIPS) in the data
        for fips, dictionary in data.items():
            # Dictionary where the key is the county code (FIPS) and the value is another dictionary
            # print('Processing:', fips)

            for key, value in dictionary.items():

                # If the value is not empty, process it
                if len(value) > 0:
                    distance_matrix = np.zeros((NUMBER_OF_VARIABLES, NUMBER_OF_VARIABLES))

                    # get the selected fips realated to the specific variable
                    selected_fips = value['FIPS'].tolist()
     
                    # filter the dataframe svi_df to only include the selected fips
                    svi_df_selected_fips = us_svi[us_svi['FIPS'].isin(selected_fips)]
                    svi_df_selected_fips = svi_df_selected_fips[VARIABLES]

                    # create a matrix
                    data_matrix = svi_df_selected_fips.to_numpy()

                    # compute the distance matrix
                    for i in range(NUMBER_OF_VARIABLES):
                        for j in range(NUMBER_OF_VARIABLES):
                            distance_matrix[i, j] = dcor.distance_correlation(data_matrix[:, i], data_matrix[:, j]) #Distance Correlations 

                    distance_matrix = distance_matrix.T + np.eye(NUMBER_OF_VARIABLES)
                    distance_matrix = 1 - distance_matrix



                else:
                    # If there is no data to compute persistence image, save an empty image
                    peristence_image = np.zeros(PERSISTENCE_IMAGE_SHAPE)
                    # np.save(os.path.join(RESULTS_DIR, key, fips), peristence_image)

                    # break
                # break
                

            # break
        
        # break

    except Exception as e:
        print(f"Error processing {state}: {e}")
        traceback.print_exc()
        continue  # Continue to the next iteration if an error occurs

print('All states processed.')

Processing states:   0%|          | 0/50 [00:00<?, ?it/s]

Processing: VT


Processing states:   2%|▏         | 1/50 [00:03<02:52,  3.53s/it]

Processing: VA


Processing states:   4%|▍         | 2/50 [00:37<17:10, 21.46s/it]

...

Processing: WA


Processing states:  98%|█████████▊| 49/50 [13:01<00:10, 10.57s/it]

Processing: TN


Processing states: 100%|██████████| 50/50 [13:17<00:00, 15.95s/it]

All states processed.
